`DataPreprocessing`

In [2]:
import pandas as pd 
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### load Data

In [18]:
PATH = '/Users/kwonkyoungmi/workspaces/workspace_Python/Git_AtAiffel/datathon/data/MentalHealth'
data = pd.read_csv(os.path.join(PATH,'df_6000(3).csv'))

In [19]:
# 결측값 확인
data.isnull().sum()

id                         0
name                       0
gender                     0
age                        0
age_group                  0
profession                 0
cgpa                       0
degree                     0
urban_rural                0
city                       0
work_study_pressure        0
work_study_satisfaction    0
financial_stress           0
sleep_duration             0
dietary_habits             0
work_study_hours           0
family_mh_history          0
suicidal_thoughts          0
depression                 0
dtype: int64

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       6000 non-null   int64  
 1   name                     6000 non-null   object 
 2   gender                   6000 non-null   int64  
 3   age                      6000 non-null   float64
 4   age_group                6000 non-null   object 
 5   profession               6000 non-null   object 
 6   cgpa                     6000 non-null   float64
 7   degree                   6000 non-null   object 
 8   urban_rural              6000 non-null   object 
 9   city                     6000 non-null   object 
 10  work_study_pressure      6000 non-null   float64
 11  work_study_satisfaction  6000 non-null   float64
 12  financial_stress         6000 non-null   float64
 13  sleep_duration           6000 non-null   object 
 14  dietary_habits          

In [21]:
data['gender'].value_counts()

gender
0    3362
1    2638
Name: count, dtype: int64

In [26]:
data['age_group'].value_counts()

age_group
46-55    1624
36-45    1431
26-35    1090
18-25    1048
56-65     807
Name: count, dtype: int64

## 데이터 탐색하기

In [23]:
data.columns

Index(['id', 'name', 'gender', 'age', 'age_group', 'profession', 'cgpa',
       'degree', 'urban_rural', 'city', 'work_study_pressure',
       'work_study_satisfaction', 'financial_stress', 'sleep_duration',
       'dietary_habits', 'work_study_hours', 'family_mh_history',
       'suicidal_thoughts', 'depression'],
      dtype='object')

In [27]:
data.describe()

,id,gender,age,cgpa,work_study_pressure,work_study_satisfaction,financial_stress,work_study_hours,family_mh_history,suicidal_thoughts,depression
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,71056.539500,0.439667,40.420333,1.514138,3.000000,2.997000,2.997833,6.286167,0.511667,0.504333,0.179500
std,40925.509348,0.496388,12.458191,3.104369,1.398091,1.399399,1.422263,3.808240,0.499906,0.500023,0.383802
min,2.000000,0.000000,18.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,35341.500000,0.000000,30.000000,0.000000,2.000000,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000
50%,72535.000000,0.000000,42.000000,0.000000,3.000000,3.000000,3.000000,6.500000,1.000000,1.000000,0.000000
75%,107340.250000,1.000000,51.000000,0.000000,4.000000,4.000000,4.000000,10.000000,1.000000,1.000000,0.000000
max,140654.000000,1.000000,60.000000,9.970000,5.000000,5.000000,5.000000,12.000000,1.000000,1.000000,1.000000


In [31]:
data.groupby('urban_rural')['city'].value_counts()

urban_rural  city         
rural        Srinagar         448
             Meerut           443
             Ludhiana         439
             Visakhapatnam    426
             Rajkot           425
             Agra             424
             Varanasi         395
urban        Ahmedabad        510
             Pune             497
             Mumbai           481
             Hyderabad        426
             Chennai          375
             Bangalore        370
             Delhi            341
Name: count, dtype: int64

In [32]:
data.urban_rural.value_counts()

urban_rural
rural    3000
urban    3000
Name: count, dtype: int64

### 조사인구특성
- urban_rural 기준으로 "gender, age_group, profession, degree"의 빈도수와 비율을 계산하여 비교 - 막대 그래프, 파이 차트, 스택 바 차트       
- 동일성 검정 (카이제곱검정)        

In [40]:
demo = ['gender','age_group','urban_rural','degree']
urban = data[data['urban_rural'] == 'urban']
rural = data[data['urban_rural'] == 'rural']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = data.copy()

target_variables = ['gender', 'age_group', 'profession', 'degree']
group_column = 'urban_rural' 


#### 동일성 검정

In [44]:
def chi2_test(dataframe, group_col, target_col, alpha=0.05):
    """
    Perform Chi-squared test of independence between two categorical variables.

    Args:
        dataframe (pd.DataFrame): The DataFrame containing the data.
        group_col (str): The column name for the grouping variable.
        target_col (str): The column name for the target variable.
        alpha (float): Significance level for the test (default is 0.05)

    if p_value < alpha: two colums are significantly related. = diffrent distribution
       p_value > alpha: two colums are not significantly related. = similar distribution
    """

    # 그룹 컬럼의 고유 값 확인 (두 개여야 함)
    unique_groups = dataframe[group_col].unique()
    group1_name, group2_name = unique_groups[0], unique_groups[1]

    # Create contingency table
    import pandas as pd
    from scipy.stats import chi2_contingency    
    contingency_table = pd.crosstab(dataframe[group_col], dataframe[target_col])

    # Perform Chi-squared test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    print(f"\n카이제곱 통계량: {chi2:.4f}")
    print(f"p-value: {p_value:.4f}")
    # 검정 결과 해석
    if p_value < alpha:
        print(f"'{group1_name}' 그룹과 '{group2_name}' 그룹에서 '{target_col}'의 분포는 다름")
    else:
        print(f"{group1_name}' 그룹과 '{group2_name}' 그룹에서 '{target_col}'의 분포는 동일")


target_variables = ['gender', 'age_group', 'profession', 'degree']

for var in target_variables:
    chi2_test(data, 'urban_rural', var)


카이제곱 통계량: 0.1522
p-value: 0.6964
rural' 그룹과 'urban' 그룹에서 'gender'의 분포는 동일

카이제곱 통계량: 6.9028
p-value: 0.1411
rural' 그룹과 'urban' 그룹에서 'age_group'의 분포는 동일

카이제곱 통계량: 45.4954
p-value: 0.1102
rural' 그룹과 'urban' 그룹에서 'profession'의 분포는 동일

카이제곱 통계량: 90.1776
p-value: 0.0000
'rural' 그룹과 'urban' 그룹에서 'degree'의 분포는 다름


In [69]:
def independent_t_test(dataframe, group_col, target_col, alpha=0.05):
    """
    두 독립 그룹 간에 특정 연속형 변수의 평균 차이를 독립표본 t-검정으로 검정합니다.
    등분산성(Levene's test)을 먼저 확인한 후, 그 결과에 따라 적절한 t-검정을 수행합니다.

    Args:
        dataframe (pd.DataFrame): 분석할 데이터프레임
        group_col (str): 두 그룹을 나누는 기준 컬럼 (예: 'urban_rural')
                         이 컬럼은 정확히 두 개의 고유한 값을 가져야 합니다
        target_col (str): 평균 차이를 검정할 연속형 변수 컬럼 (예: 'age', 'cgpa')
        alpha (float, optional): 유의수준 (기본값: 0.05)
    """
    print(f"\n--- '{group_col}' 그룹 간 '{target_col}' 평균 차이 독립표본 t-검정 ---\n")

    from scipy import stats

    # 그룹 컬럼의 고유 값 확인
    unique_groups = dataframe[group_col].unique()

    group1_name, group2_name = unique_groups[0], unique_groups[1]

    # 각 그룹별 데이터 추출 및 결측치 제거
    group1_data = dataframe[dataframe[group_col] == group1_name][target_col].dropna()
    group2_data = dataframe[dataframe[group_col] == group2_name][target_col].dropna()

    # 그룹별 기술 통계량 (평균, 표준편차) 확인
    print(f"  '{group1_name}' 그룹 '{target_col}' 평균: {group1_data.mean():.2f}, (표준편차: {group1_data.std():.2f}, N={len(group1_data)})")
    print(f"  '{group2_name}' 그룹 '{target_col}' 평균: {group2_data.mean():.2f}, (표준편차: {group2_data.std():.2f}, N={len(group2_data)})")

    # 1. 등분산성 검정 (Levene's test)
    # 귀무가설 (H0): 두 그룹의 분산이 동일하다.
    # 대립가설 (H1): 두 그룹의 분산이 다르다.
    levene_stat, levene_p_value = stats.levene(group1_data, group2_data)
    print(f"\n  Levene's Test (등분산성 검정) => 통계량: {levene_stat:.4f}, p-value: {levene_p_value:.4f}")

    # 2. 독립표본 t-검정 수행
    # p-value가 유의수준(0.05)보다 작으면 귀무가설 기각
    if levene_p_value > alpha: # 등분산 가정이 충족될 경우 (p > alpha)
        # 등분산 가정 하에 t-검정
        t_stat, p_value = stats.ttest_ind(group1_data, group2_data, equal_var=True)
        print("  --> 분산이 동일하다는 가정 하에 t-검정 (Equal variances assumed)")
    else: # 등분산 가정이 충족되지 않을 경우 (p <= alpha)
        # 등분산 가정을 하지 않는 웰치(Welch)의 t-검정
        t_stat, p_value = stats.ttest_ind(group1_data, group2_data, equal_var=False)
        print("  --> 분산이 동일하지 않다는 가정 하에 t-검정 (Welch's t-test)")

    if p_value < alpha:
        print(f"\n t-통계량: {t_stat:.4f}, p-value ({p_value:.4f}) : '{group1_name}' 그룹과 '{group2_name}' 그룹의 '{target_col}' 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print(f"\n t-통계량: {t_stat:.4f}, p-value ({p_value:.4f}) : '{group1_name}' 그룹과 '{group2_name}' 그룹의 '{target_col}' 평균이 통계적으로 유의미하게 다르다고 말하기 어려움")


### 라이프스타일 비교 - urban_rural 그룹별 sleep_duration, dietary_habits, work_study_hours

- 라이프스타일 특성  : 평균,중앙값,표준분포 및 분포  (범주형 vs 수치형 : Box plot, Violin plot)        

- 라이프스타일 검정 (동일성)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       6000 non-null   int64  
 1   name                     6000 non-null   object 
 2   gender                   6000 non-null   int64  
 3   age                      6000 non-null   float64
 4   age_group                6000 non-null   object 
 5   profession               6000 non-null   object 
 6   cgpa                     6000 non-null   float64
 7   degree                   6000 non-null   object 
 8   urban_rural              6000 non-null   object 
 9   city                     6000 non-null   object 
 10  work_study_pressure      6000 non-null   float64
 11  work_study_satisfaction  6000 non-null   float64
 12  financial_stress         6000 non-null   float64
 13  sleep_duration           6000 non-null   object 
 14  dietary_habits          

In [ ]:
# 라이프 스타일은 다르게 나오는 것이 좋음

In [54]:
target_variables = ['sleep_duration', 'dietary_habits']

for var in target_variables:
    chi2_test(df, 'urban_rural', var)


카이제곱 통계량: 8.5972
p-value: 0.0136
'rural' 그룹과 'urban' 그룹에서 'sleep_duration'의 분포는 다름

카이제곱 통계량: 0.7817
p-value: 0.6765
rural' 그룹과 'urban' 그룹에서 'dietary_habits'의 분포는 동일


In [70]:
independent_t_test(df, 'urban_rural', 'work_study_hours')


--- 'urban_rural' 그룹 간 'work_study_hours' 평균 차이 독립표본 t-검정 ---

  'rural' 그룹 'work_study_hours' 평균: 6.38, (표준편차: 3.80, N=3000)
  'urban' 그룹 'work_study_hours' 평균: 6.19, (표준편차: 3.82, N=3000)

  Levene's Test (등분산성 검정) => 통계량: 0.7897, p-value: 0.3742
  --> 분산이 동일하다는 가정 하에 t-검정 (Equal variances assumed)

 t-통계량: 1.8954, p-value (0.0581) : 'rural' 그룹과 'urban' 그룹의 'work_study_hours' 평균이 통계적으로 유의미하게 다르다고 말하기 어려움


### 스트레스지표 비교 - urban_rural 그룹별 work_study_pressure, financial_stress 
- 지역별 스트레스 특성 : 평균 및 분포 비교      
- 스트레스 검정 (동일성)

In [72]:
# 스트레스 지표는 다르게 나오는 것이 좋음
target_variables = ['work_study_pressure', 'financial_stress', 'work_study_satisfaction']

for var in target_variables:
    independent_t_test(df, 'urban_rural', var)


--- 'urban_rural' 그룹 간 'work_study_pressure' 평균 차이 독립표본 t-검정 ---

  'rural' 그룹 'work_study_pressure' 평균: 3.01, (표준편차: 1.40, N=3000)
  'urban' 그룹 'work_study_pressure' 평균: 2.99, (표준편차: 1.40, N=3000)

  Levene's Test (등분산성 검정) => 통계량: 0.0559, p-value: 0.8131
  --> 분산이 동일하다는 가정 하에 t-검정 (Equal variances assumed)

 t-통계량: 0.4247, p-value (0.6710) : 'rural' 그룹과 'urban' 그룹의 'work_study_pressure' 평균이 통계적으로 유의미하게 다르다고 말하기 어려움

--- 'urban_rural' 그룹 간 'financial_stress' 평균 차이 독립표본 t-검정 ---

  'rural' 그룹 'financial_stress' 평균: 2.99, (표준편차: 1.42, N=3000)
  'urban' 그룹 'financial_stress' 평균: 3.01, (표준편차: 1.42, N=3000)

  Levene's Test (등분산성 검정) => 통계량: 0.0252, p-value: 0.8738
  --> 분산이 동일하다는 가정 하에 t-검정 (Equal variances assumed)

 t-통계량: -0.4084, p-value (0.6830) : 'rural' 그룹과 'urban' 그룹의 'financial_stress' 평균이 통계적으로 유의미하게 다르다고 말하기 어려움

--- 'urban_rural' 그룹 간 'work_study_satisfaction' 평균 차이 독립표본 t-검정 ---

  'rural' 그룹 'work_study_satisfaction' 평균: 3.02, (표준편차: 1.40, N=3000)
  'urban' 그룹 'work_study_s

### 정신건강지표 비교 - urban_rural 그룹별 suidial_thoughts, depression
- 정신건강지표 특성 : 발생률(비율)비교, study_work_satisfaction의 평균 및 분포를 비교 (범주형 vs 범주형 : stacked bar chart)          
- 정신건강 검정(동일성)

In [52]:
# 정신건강 지표는 다르게 나오는 것이 좋음 - 가설은 도시가 더 좋을 것이다.
# 스트레스 지표는 다르게 나오는 것이 좋음
target_variables = ['suicidal_thoughts', 'depression']

for var in target_variables:
    chi2_test(df, 'urban_rural', var)


카이제곱 통계량: 0.0167
p-value: 0.8973
rural' 그룹과 'urban' 그룹에서 'suicidal_thoughts'의 분포는 동일

카이제곱 통계량: 0.5477
p-value: 0.4593
rural' 그룹과 'urban' 그룹에서 'depression'의 분포는 동일


### 상관 분석
- urban_rural 그룹별 : sleep_duration, dietary_habits(수치화 후),work_study_hours 와 depression, suidial_thoughts, work_study_satisfaction 간의 상관관계              
- 전체 데이터에서의 상관관계                